This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/stochastic.html).  I recommend having both windows open, side-by-side!


In [ ]:
import mpld3
import numpy as np
from IPython.display import display
from matplotlib import pyplot as plt
from pydrake.all import (
    AddRandomInputs,
    DiagramBuilder,
    LeafSystem,
    PyPlotVisualizer,
    RandomDistribution,
    Simulator,
)
from pydrake.examples import VanDerPolOscillator

from underactuated import running_as_notebook
from underactuated.pyplot_utils import AdvanceToAndVisualize, HistogramVisualizer

if running_as_notebook:
    mpld3.enable_notebook()

# A Bistable System w/ Gaussian Noise

In [ ]:
def dynamics(x, w):
    return x - x**3 + w


class SimpleStochasticSystem(LeafSystem):
    def __init__(self, num_particles):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort(
            "noise", num_particles, RandomDistribution.kGaussian
        )
        index = self.DeclareDiscreteState(num_particles)
        self.h = 0.1
        self.sigma = 0.3
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=self.h, offset_sec=0, update=self.Update
        )
        self.DeclareStateOutputPort("state", index)

    def Update(self, context, discrete_state):
        x = context.get_discrete_state_vector().value()
        w = self.EvalVectorInput(context, 0).value()
        xn = x + self.h * dynamics(x, self.sigma * w)
        discrete_state.get_mutable_vector().SetFromVector(xn)


def bistable_demo():
    builder = DiagramBuilder()

    num_particles = 1000
    num_bins = 100
    xlim = [-2, 2]
    ylim = [-1, 3.5]
    draw_time_step = 0.25
    visualizer = builder.AddSystem(
        HistogramVisualizer(num_particles, num_bins, xlim, ylim, draw_time_step)
    )
    x = np.linspace(xlim[0], xlim[1], 100)
    visualizer.ax.plot(x, dynamics(x, 0), "k", linewidth=2)

    sys = builder.AddSystem(SimpleStochasticSystem(num_particles))
    builder.Connect(sys.get_output_port(), visualizer.get_input_port())

    AddRandomInputs(0.1, builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_integrator().set_fixed_step_mode(True)
    simulator.get_mutable_integrator().set_maximum_step_size(0.1)

    AdvanceToAndVisualize(simulator, visualizer, 20.0, 1.0)


bistable_demo()

In [ ]:
# If you want physical intuition, you can think of this system as a particle doing gradient descent on the potential function U(x)

xs = np.linspace(-1.5, 1.5, 101)
plt.plot(xs, -0.5 * xs**2 + 0.25 * xs**4)
plt.xlabel("x")
plt.ylabel("U(x)")
display(mpld3.display())

Now let's consider the time-reversed version of the same system; it has a stable fixed-point at the origin

In [ ]:
def dynamics(x, w):
    # Use clip so that x doesn't exponentially diverge to infinity
    return np.clip(-x + x**3 + w, -100, 100)


def time_reversed_demo():
    builder = DiagramBuilder()

    num_particles = 10000 if running_as_notebook else 10
    num_bins = 100
    xlim = [-2, 2]
    ylim = [-1, 3.5]
    draw_time_step = 0.25
    visualizer = builder.AddSystem(
        HistogramVisualizer(num_particles, num_bins, xlim, ylim, draw_time_step)
    )
    x = np.linspace(xlim[0], xlim[1], 100)
    visualizer.ax.plot(x, dynamics(x, 0), "k", linewidth=2)

    sys = builder.AddSystem(SimpleStochasticSystem(num_particles))
    sys.sigma = 1.5
    builder.Connect(sys.get_output_port(), visualizer.get_input_port())

    AddRandomInputs(0.1, builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_integrator().set_fixed_step_mode(True)
    simulator.get_mutable_integrator().set_maximum_step_size(0.1)

    AdvanceToAndVisualize(simulator, visualizer, 20.0, 1.0)


time_reversed_demo()

# The Stochastic Van der Pol Oscillator

TODO(russt): Port the visualization to meshcat.

In [ ]:
class VanDerPolParticles(LeafSystem):
    def __init__(self, num_particles, mu=1.0):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort(
            "noise", num_particles, RandomDistribution.kGaussian
        )
        state_index = self.DeclareContinuousState(num_particles, num_particles, 0)
        self.DeclareStateOutputPort("state", state_index)
        self.num_particles = num_particles
        self.mu = mu

    # TODO(russt):  SetRandomState to  [-0.1144;2.0578] + 0.01*randn(...)

    def DoCalcTimeDerivatives(self, context, derivatives):
        # TODO(russt):  Update this to get_position/velocity once those are
        # bound.
        x = context.get_continuous_state_vector().CopyToVector()
        q = x[: self.num_particles]
        qdot = x[self.num_particles :]
        w = self.EvalVectorInput(context, 0).CopyToVector()
        qddot = -self.mu * (q * q - 1) * qdot - q + 0.5 * w
        derivatives.get_mutable_vector().SetFromVector(np.concatenate((qdot, qddot)))


# Note: This is a candidate for moving to a more central location.
class Particle2DVisualizer(PyPlotVisualizer):
    def __init__(self, num_particles, xlim, ylim, draw_time_step):
        PyPlotVisualizer.__init__(self, draw_time_step, show=False, figsize=(6, 7))
        self.DeclareVectorInputPort("x", 2 * num_particles)
        self.num_particles = num_particles
        self.ax.set_xlim(xlim)
        self.ax.set_ylim(ylim)
        zero = [0] * num_particles
        (self.lines,) = self.ax.plot(zero, zero, "b.")

    def draw(self, context):
        xy = self.EvalVectorInput(context, 0).CopyToVector()
        self.lines.set_xdata(xy[: self.num_particles])
        self.lines.set_ydata(xy[self.num_particles :])
        self.ax.set_title("t = " + str(context.get_time()))


def stochastic_van_der_pol():
    builder = DiagramBuilder()

    num_particles = 5000
    xlim = [-3, 3]
    ylim = [-3.5, 3.5]
    draw_time_step = 0.5
    sys = builder.AddSystem(VanDerPolParticles(num_particles))
    visualizer = builder.AddSystem(
        Particle2DVisualizer(num_particles, xlim, ylim, draw_time_step)
    )
    builder.Connect(sys.get_output_port(0), visualizer.get_input_port(0))
    AddRandomInputs(0.1, builder)

    # Plot nominal limit cycle.
    vdp = VanDerPolOscillator()
    limit_cycle = vdp.CalcLimitCycle()
    visualizer.ax.plot(limit_cycle[0, :], limit_cycle[1, :], "k")

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.set_publish_every_time_step(False)
    simulator.get_mutable_integrator().set_fixed_step_mode(True)
    simulator.get_mutable_integrator().set_maximum_step_size(0.1)

    # Set initial conditions around a known point on the limit cycle
    x0 = 0.1 * np.random.randn(2 * num_particles)
    x0[:num_particles] += -0.1144
    x0[num_particles:] += 2.0578
    simulator.get_mutable_context().SetContinuousState(x0)

    AdvanceToAndVisualize(simulator, visualizer, 30.0, 1.0)
    simulator.AdvanceTo(1000 if running_as_notebook else 1)
    visualizer.draw(visualizer.GetMyContextFromRoot(simulator.get_context()))
    display(mpld3.display(visualizer.fig))


stochastic_van_der_pol()